In [1]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# https://developers.google.com/workspace/gmail/api/quickstart/python?hl=es-419
def get_creds():
  """Shows basic usage of the Gmail API.
  Lists the user's Gmail labels.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  return creds

def get_gmail_service(creds = None):
  if creds is None:
    creds = get_creds()
  return build("gmail", "v1", credentials=creds)

creds = get_creds()
service = get_gmail_service(creds)

In [14]:

def list_labels(service = None, creds = None):
  try:
    # Call the Gmail API
    if service is None: 
        service = get_gmail_service(creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("No labels found.")
      return
    print("Labels:")
    for label in labels:
      print(label["name"])

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

list_labels(service=service, creds=creds)

Labels:
CHAT
SENT
INBOX
IMPORTANT
TRASH
DRAFT
SPAM
CATEGORY_FORUMS
CATEGORY_UPDATES
CATEGORY_PERSONAL
CATEGORY_PROMOTIONS
CATEGORY_SOCIAL
YELLOW_STAR
STARRED
UNREAD
Camper
Notes
irph
bsm
llogaters
Documentos
Facturas
Comunidad


In [21]:
# https://developers.google.com/workspace/gmail/api/reference/rest?hl=es-419

def list_messages(maxResults=10,service = None, creds = None):
  try:
    # Call the Gmail API
    if service is None: 
        service = get_gmail_service(creds)
    results = service.users().messages().list(userId="me", maxResults=maxResults).execute()
    messages = results.get("messages", [])

    if not messages:
      print("No messages found.")
      return
    print("Items:")
    # for item in messages:
    print(results)

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

list_messages(10,service=service, creds=creds)

Items:
{'messages': [{'id': '19b99943f7fd623c', 'threadId': '19b99943f7fd623c'}, {'id': '19b98afd6af87b83', 'threadId': '19b98afd6af87b83'}, {'id': '19b9895d0a73862b', 'threadId': '19b9892c514c3395'}, {'id': '19b9892c514c3395', 'threadId': '19b9892c514c3395'}, {'id': '19b9864943f0295d', 'threadId': '19b9864943f0295d'}, {'id': '19b985808f02fda5', 'threadId': '19b985808f02fda5'}, {'id': '19b976c7d0b6d089', 'threadId': '19b976c7d0b6d089'}, {'id': '19b9739aa5f29a7e', 'threadId': '19b9739aa5f29a7e'}, {'id': '19b9739780e7d820', 'threadId': '19b9739780e7d820'}, {'id': '19b96c7b08c4d2bc', 'threadId': '19b96c7b08c4d2bc'}], 'nextPageToken': '18133380041699689345', 'resultSizeEstimate': 201}


In [8]:
def get_message(id, service = None, creds = None):
  try:
    # Call the Gmail API
    if service is None: 
        service = get_gmail_service(creds)
    results = service.users().messages().get(userId="me", id=id).execute()
    # messages = results.get("messages", [])

    # if not messages:
    #   print("No messages found.")
    #   return
    # print("Labels:")
    # for item in messages:
    print(results)

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

get_message("19b940f6ac06bd5c", service, creds)

{'id': '19b940f6ac06bd5c', 'threadId': '19b940f6ac06bd5c', 'labelIds': ['CATEGORY_UPDATES', 'INBOX'], 'snippet': 'Hi there, Llama Enthusiasts! Welcome to 2026! We&#39;re kicking off the new year with exciting developments in document AI workflows. This week brings powerful Agent Workflow integrations with the', 'payload': {'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'germanztz@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:6918:a5a2:b0:3f1:44be:4d2b with SMTP id bj34csp1538592ysb;        Tue, 6 Jan 2026 08:06:37 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IGD2UsF2RyS9fGQhYn8JYtX/jLpCI5hhN+BWzXpkZtguH7QwDdPB0R27281fOj4oBkZgwiJ'}, {'name': 'X-Received', 'value': 'by 2002:a17:903:2344:b0:298:2637:800b with SMTP id d9443c01a7336-2a3e2deab5cmr37227445ad.31.1767715597006;        Tue, 06 Jan 2026 08:06:37 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1767715596; cv=none;      